We install here the required dependencies

In [1]:
!python -m pip install kuzu

In [2]:
import kuzu

db = kuzu.Database('./kuzudb_elt')

conn = kuzu.Connection(db)

load_json_extension = """
INSTALL json;
LOAD EXTENSION json;
"""

conn.execute(load_json_extension)

Let's drop the old edges and then nodes (if they exist)

In [3]:
SHOW_EDGE_TABLES = """
CALL SHOW_TABLES() WHERE type='REL' RETURN *;
"""
result_set = conn.execute(SHOW_EDGE_TABLES)
while result_set.has_next():
    table_name = result_set.get_next()[0]
    print(f"DROPPING {table_name}")
    conn.execute(f"DROP TABLE {table_name};")

result_set.close()

SHOW_NODE_TABLES = """
CALL SHOW_TABLES() WHERE type='NODE' RETURN *;
"""
result_set = conn.execute(SHOW_NODE_TABLES)
while result_set.has_next():
    table_name = result_set.get_next()[0]
    print(f"DROPPING {table_name}")
    conn.execute(f"DROP TABLE {table_name};")

result_set.close()

Now we load the nodes.

In [4]:
with open('dml_statements/elt_kuzudb_node_schema.sql') as f:
    creation_lines = [line for line in f]

with open('dml_statements/elt_kuzudb_node_import.sql') as f:
    import_lines = [line for line in f]
    

#do not forget the primary keys for seeds

    
for line in creation_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

for line in import_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

Executing statement CREATE NODE TABLE IMDB_BUSINESS_CAST(cast_id STRING, description STRUCT(category STRING, characters STRING)[], job STRING, PRIMARY KEY (cast_id));

Executing statement CREATE NODE TABLE IMDB_BUSINESS_NAME(nconst STRING, primaryName STRING, birthYear INT64, deathYear INT64, primaryProfession STRING[], PRIMARY KEY (nconst));

Executing statement CREATE NODE TABLE IMDB_BUSINESS_TITLE(tconst STRING, titleType STRING, primaryTitle STRING, originalTitle STRING, isAdult BOOLEAN, startYear INT64, endYear INT64, runtimeMinutes INT64, genres STRING[], averageRating FLOAT, numVotes INT64, seasonNumber INT64, episodeNumber INT64, PRIMARY KEY (tconst));

Executing statement CREATE NODE TABLE IMDB_BUSINESS_TITLE_ALIAS(alias_id STRING, title STRING, region STRING, language STRING,  types STRING[], attributes STRING[], isOriginalTitle BOOLEAN, PRIMARY KEY (alias_id));
Executing statement COPY IMDB_BUSINESS_CAST FROM "./export_elt/node_gold_business_cast.json"

Executing statement C

Now we load the relationships.

In [ ]:
with open('dml_statements/elt_kuzudb_rel_schema.sql') as f:
    creation_lines = [line for line in f]

with open('dml_statements/elt_kuzudb_rel_import.sql') as f:
    import_lines = [line for line in f]
    

#do not forget the primary keys for seeds
    
for line in creation_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

for line in import_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

Executing statement CREATE REL TABLE  IMDB_BUSiNESS_NAME_KNOWN_FOR_TITLE(FROM IMDB_BUSINESS_NAME TO IMDB_BUSINESS_TITLE);

Executing statement CREATE REL TABLE  IMDB_BUSiNESS_TITLE_ALIAS_ALIASES_TITLE(FROM IMDB_BUSINESS_TITLE_ALIAS TO IMDB_BUSINESS_TITLE);

Executing statement CREATE REL TABLE  IMDB_BUSiNESS_TITLE_HAS_DIRECTOR_NAME(FROM IMDB_BUSINESS_TITLE TO IMDB_BUSINESS_NAME);

Executing statement CREATE REL TABLE  IMDB_BUSiNESS_TITLE_HAS_PARENT_TITLE(FROM IMDB_BUSINESS_TITLE TO IMDB_BUSINESS_TITLE);

Executing statement CREATE REL TABLE  IMDB_BUSiNESS_TITLE_HAS_WRITER_NAME(FROM IMDB_BUSINESS_TITLE TO IMDB_BUSINESS_NAME);
Executing statement COPY IMDB_BUSiNESS_NAME_KNOWN_FOR_TITLE FROM "./export_elt/edge_gold_business_name_known_for_title.json"

Executing statement COPY IMDB_BUSiNESS_TITLE_ALIAS_ALIASES_TITLE FROM "./export_elt/edge_gold_business_title_alias_aliases_title.json"

Executing statement COPY IMDB_BUSiNESS_TITLE_HAS_DIRECTOR_NAME FROM "./export_elt/edge_gold_business_titl

RuntimeError: Copy exception: Found NULL, which violates the non-null constraint of the primary key column.